In [48]:
word_vectors = 0
context_vectors = 0
clf = 0
idf = 0
df = 0
ndf = 0

In [5]:
word_vectors = load_vectors("/Users/fedorsimonov/Documents/Study/Programming/DeepLearningNLP/GoogleNews-vectors-negative300.bin")
print ("Word vectors loaded")
context_vectors = word_vectors

Word vectors loaded


In [58]:
# -*- coding: utf-8 -*-

import random
import numpy as np
from russe.w2v.utils import load_vectors, load_word2vec_format
from collections import Counter, OrderedDict
import pandas as pd
import re
from sys import stdout, stderr
import gzip
import pymorphy2
import pymorphy2.tokenizers
import re
from sklearn.base import BaseEstimator, ClassifierMixin
import subprocess

class MaxProbClf(BaseEstimator, ClassifierMixin):
    def __init__(self, sense_df, vw, vc):
        self.sense_df = sense_df
        self.vw = vw
        self.vc = vc
           
    def logprob(self, ctx, vsense):
        vc = self.vc
        vctx = vc.syn0norm[vc.vocab[ctx].index]
        return np.log(1.0 / (1.0 + np.exp(-np.dot(vctx,vsense))))
    
    def fit(self, X, y):
        pass
    
    
    def get_senses(self, homo, cache):
        if homo not in cache:
            sense_df = self.sense_df
            senses, svecs = [], []
            contexts = []
            homos = homo.split('|')
            for i, row in sense_df[sense_df.word.isin(homos)].iterrows():
                svec = build_svec(row.word, row.context, self.vw)
                sense = row.id #'%s_%d' % (row.word,row.id) if '|' not in homo else row.word
                context = row.context
                if len(svec):
                    senses.append(sense)
                    svecs.append(svec)
                    contexts.append(context)
            svecs = normalizeRows(np.array(svecs))
            cache[homo] = (senses, svecs, contexts)
            #for t, vec in enumerate(svecs):
            #        print t, homo, ':'
            #        for cont in self.vw.most_similar(positive=[vec]):
            #            print '\t', cont
            #for q in range(len(svecs)):
            #    for p in range(q + 1, len(svecs)):
            #        print p, q, 1.0 / (1.0 + np.exp(-np.dot(svecs[p],svecs[q])))
            
        return cache[homo]
    
    
    def predict(self, cdf, gamma):
        y = []
        cache = dict()
        for i in xrange(len(cdf)):
            l, homo, r, id = takeI(cdf, i)
            senses, svecs, contexts = self.get_senses(homo,cache)
            cntSenses = len(senses)
            s = [0.0 for n in xrange(cntSenses)]
            words = [word for word in l+r if word in self.vc and word in self.vw]
            isRelevantCtx = False
            print ' '.join(l + list(homo) + r)
            maxLp = 0
            for word in words:
                lp = [self.logprob(word, svecs[j]) for j in xrange(cntSenses)]
                
                for i in xrange(cntSenses):
                    for j in xrange(i + 1, cntSenses):
                        diff = lp[i] - lp[j]
                        if np.abs(diff) > maxLp:
                            maxLp = np.abs(diff)
                            
            for word in words:
                lp = [self.logprob(word, svecs[j]) for j in xrange(cntSenses)]
                
                vc = self.vc
                vctx = vc.syn0norm[vc.vocab[word].index]
                dots = [np.dot(vctx, svecs[j]) for j in xrange(cntSenses)]
                ans = []
                for j in range(len(lp)):
                    ans.append('P(%s|%s_%d)=%.4f' % (word, homo, j, 1.0 / (1.0 + np.exp(-dots[j]))))
                print ' '.join(ans)
                for i in xrange(cntSenses):
                    for j in xrange(i + 1, cntSenses):
                        diff = lp[i] - lp[j]
                        if np.abs(diff) > (maxLp * gamma):
                            isRelevantCtx = True
                if isRelevantCtx:
                    for j in xrange(cntSenses):
                        s[j] += lp[j]
            print s
            ind = np.argsort(s)
            if len(senses) != 0:
                sense = senses[ind[-1]]
                #print 'Predicted sense - %d\n--------------' % sense
                context = contexts[ind[-1]]
                y.append((sense, id))
                cdf.loc[id, 'predict_sense_ids'] = int(sense)
                cdf.loc[id, 'predict_related'] = context
                if id % 1000 == 0:
                    print "%.2f%% completed" % (float(id) / float(len(cdf)))
        return y

# Implement a function that normalizes each row of a matrix to have unit length
def normalizeRows(x):
    mark = False
    if len(x.shape) == 1:
        mark = True
        x = x.reshape((1, x.shape[0]))
    r, c = x.shape
    t = np.sqrt(np.sum(x**2, axis = 1))
    x /= t.reshape(t.shape[0], 1)
    if (mark):
        x = x.reshape((x.shape[1],))
    return x

def takeI(cdf, i):
    context = cdf['context'][i]
    borders = cdf['target_position'][i].split(',')
    borders = [int(x) for x in borders]
    L = context[:borders[0]].split()
    word = cdf['target'][i]
    R = context[borders[1]:].split()
    return L, word, R, i

_morph = pymorphy2.MorphAnalyzer()
rr = re.compile(u'[a-zа-яё]')

def tokenize(sent):
    u"""
    Simple tokenizer - lowercases everything and tokenizes using pymorphy2, removes all punctuation tokens.
    >>> ' * '.join( tokenize(u'Для') ) == u'для'
    True
    >>> ' * '.join(tokenize(u'Для стрельбы-стрельбы, из Арбалета? использовались болты — особые Арбалетные стрелы!')) == u'для * стрельбы-стрельбы * из * арбалета * использовались * болты * особые * арбалетные * стрелы'
    True
    """

    sent = sent.lower()
    words = [x for x in pymorphy2.tokenizers.simple_word_tokenize(sent) if rr.search(x) is not None]
    return words


def has_normal_form(word, normal):
    u"""
    Returns whether the word has specified normal form.
    >>> has_normal_form(u'лейки',u'лейка')
    True
    >>> has_normal_form(u'лейки',u'лейки')
    False
    >>> has_normal_form(u'берег',u'беречь')
    True
    >>> has_normal_form(u'берег',u'берег')
    True
    """
    for p in _morph.parse(word):
        if p.normal_form==normal:
            return True
    return False

def parse_neigh(line, K=None, return_dists=False):
    neigh = [(x.split(':')[0],float(x.split(':')[1])) for x in (line.replace(" ", "")).split(',')]
    neigh = sorted(neigh, key=lambda x:x[1],reverse=True)
    neigh_part = neigh[:K] if K is not None else neigh
    if return_dists:
        return neigh_part
    else:
        return [x[0] for x in neigh_part]

def build_svec(word, neigh, vw):
    neigh = parse_neigh(neigh, return_dists=False)
    nlog = []
    selected_neigh = []
    for x in neigh:
        if has_normal_form(x,word):
            nlog.append('*%s' % x)
            continue
        if x not in vw:
            nlog.append('-%s' % x)
            continue
        else:
            return vw[x]
        sense_cnt = len(idf[idf.word==x])
        if (sense_cnt>1):
            nlog.append('?%ds?%s' % (sense_cnt, x) )
            continue

        nlog.append('+%s' % x)
        selected_neigh.append(x.strip())
    #print >> stderr, '%s (%s)' % (word, ' '.join(nlog))
    if len(selected_neigh) != 0:
        svec = np.average(vw[selected_neigh],axis=0)
        #print >> stderr, ' '.join(['%s %.2lf' % p for p in vw.most_similar(positive=[svec])])
    else:
        svec = np.zeros(vw.syn0[0].shape)
    return svec

def delete_substring(name, str):
    i = name.rfind(str)
    if i != -1:
        return name[0:i]
    else:
        return name

def name_of_file(name):
    i = name.rfind('/')
    if i != -1:
        return name[i + 1 :]
    else:
        return name

In [59]:
FSET = './data/TWSI_dev.csv'
FINV = './data/Inventory-TWSI-2.csv' # './data/Inventory-59g-agagram-alpha-15.csv.gz', './data/Inventory-59g-agagram-alpha-20.csv.gz', './data/Inventory-59g-agagram-alpha-75.csv.gz'
gamma = 0.002

In [60]:
df = pd.read_csv(FSET, sep = '\t', encoding='utf8')
df.loc[:, 'predict_sense_ids'] = np.array([0] * len(df), dtype='int32')
print ("Test set loaded")

Test set loaded


In [61]:
idf = pd.read_csv(FINV, sep = '\t', encoding='utf8', names=['word', 'id', 'context'])
if (idf.loc[0,'word'] == 'word'):
    idf = idf.loc[1:]
print ("Inventory loaded")

Inventory loaded


In [62]:
for word in set(df.loc[:, 'target']):
    print word

coach
chain
dance
child
resistance
code
captain
competition
environment
charge
program
division
advantage
fall
difference
condition
magic
level
solution
disc
ride
programme
literature
sign
second
cost
investment
folk
settlement
conduct
principal
movement
body
degree
leg
makeup
commercial
water
protection
studio
aid
path
legend
box
voice
study
experience
trial
advertising
defeat
action
honor
criticism
love
family
claim
total
unit
plot
use
estate
visit
memory
strike
minor
entrance
door
club
instrumental
visual
adult
effort
account
challenge
work
soul
male
heart
figure
share
minimum
sense
native
end
goal
travel
feature
answer
establishment
profile
map
product
designer
farm
philosophy
spot
ball
plane
blood
structure
faculty
date
law
purchase
counter
element
responsibility
representation
talk
producer
move
mission
damage
brain
paper
existence
cold
style
organisation
facility
feel
window
production
mail
criminal
crash
combination
material
scene
foot
day
association
successor
opera
drop
mode


In [63]:
clf = MaxProbClf(idf, word_vectors, context_vectors)

In [64]:
ndf = pd.concat([df.loc[df['target'] == 'language'], df.loc[df['target'] == 'square'], df.loc[df['target'] == 'face'], df.loc[df['target'] == 'match']], ignore_index=True)

In [12]:
clf.predict(df, 2)
print "%.2f%% completed" % float(100)
print ("Senses predicted")

0.00% completed
0.16% completed
0.32% completed
0.49% completed
0.65% completed
0.81% completed
0.97% completed
100.00% completed
Senses predicted


In [65]:
clf.predict(ndf, 0)
print "%.2f%% completed" % float(100)
print ("Senses predicted")
ndf

Hale &amp; Salamanca ( 2001 ) classifies the Sumu languages into a northern composed of the Twahka and Panamahka dialects and southern Sumu consisting of the Ulwa l a n g u a g e .
P(Hale|language_0)=0.4961 P(Hale|language_1)=0.5263
P(&amp;|language_0)=0.5077 P(&amp;|language_1)=0.4790
P(Salamanca|language_0)=0.5113 P(Salamanca|language_1)=0.5105
P(classifies|language_0)=0.5106 P(classifies|language_1)=0.5283
P(the|language_0)=0.5003 P(the|language_1)=0.5209
P(languages|language_0)=0.6225 P(languages|language_1)=0.5450
P(into|language_0)=0.5032 P(into|language_1)=0.5143
P(northern|language_0)=0.5610 P(northern|language_1)=0.5131
P(composed|language_0)=0.5362 P(composed|language_1)=0.5000
P(the|language_0)=0.5003 P(the|language_1)=0.5209
P(dialects|language_0)=0.6810 P(dialects|language_1)=0.5289
P(southern|language_0)=0.5622 P(southern|language_1)=0.5198
P(consisting|language_0)=0.5038 P(consisting|language_1)=0.5086
P(the|language_0)=0.5003 P(the|language_1)=0.5209
[-8.806885327338074

,context_id,target,target_pos,target_position,gold_sense_ids,predict_sense_ids,golden_related,predict_related,context
0,10061196,language,n,"163,171",1,1,"dialect:20, speech:19, vocabulary:16, word:13,...","dialect:20, speech:19, vocabulary:16, word:13,...",Hale &amp; Salamanca ( 2001 ) classifies the S...
1,10230031,language,n,"14,22",1,1,"dialect:20, speech:19, vocabulary:16, word:13,...","dialect:20, speech:19, vocabulary:16, word:13,...",The principal language is Ndebele which is spo...
2,10695828,language,n,"43,51",1,1,"dialect:20, speech:19, vocabulary:16, word:13,...","dialect:20, speech:19, vocabulary:16, word:13,...",The questions have a minimal dependence on lan...
3,11513082,language,n,"94,102",1,1,"dialect:20, speech:19, vocabulary:16, word:13,...","dialect:20, speech:19, vocabulary:16, word:13,...",The I Tatti Renaissance Library presents key M...
4,1159861,language,n,"69,77",1,1,"dialect:20, speech:19, vocabulary:16, word:13,...","dialect:20, speech:19, vocabulary:16, word:13,...","However , French is only spoken by 10 % of the..."
5,38533133,language,n,"287,295",2,2,"computer language:10, programming language:7, ...","computer language:10, programming language:7, ...","In this case , differences were due to US Gold..."
6,47587481,language,n,"110,118",2,2,"computer language:10, programming language:7, ...","computer language:10, programming language:7, ...",This allows a programmer to use the DSL for pr...
7,48382841,language,n,"78,86",2,2,"computer language:10, programming language:7, ...","computer language:10, programming language:7, ...","By comparison , a disassembler translates an e..."
8,50160491,language,n,"36,44",2,2,"computer language:10, programming language:7, ...","computer language:10, programming language:7, ...",In the Verilog hardware description language s...
9,50419061,language,n,"2,10",2,2,"computer language:10, programming language:7, ...","computer language:10, programming language:7, ...",A language supporting reflection provides a nu...


In [23]:
idf.loc[idf['word'] == 'match']

,word,id,context
1251,match,1,"contest:37, game:26, competition:22, bout:13, ..."
1252,match,2,"equal:16, duplicate:7, double:5, copy:5, compl..."


In [102]:
df.loc[df['target'] == 'square']

,context_id,target,target_pos,target_position,gold_sense_ids,predict_sense_ids,golden_related,predict_related,context
5230,5606267,square,n,"91,98",1,2,"quadrate:10, four-side:10, equal-side:10, quad...","plaza:4, town square:2, center:2, market squar...","By 11 September , up to 50,000 people had join..."
5231,8923119,square,n,"160,166",1,2,"quadrate:10, four-side:10, equal-side:10, quad...","plaza:4, town square:2, center:2, market squar...",Captain Sternn : Based on the original story b...
5232,11444992,square,n,"73,79",1,1,"quadrate:10, four-side:10, equal-side:10, quad...","quadrate:10, four-side:10, equal-side:10, quad...",The evolution of its design was an expansion o...
5233,17674325,square,n,"24,30",1,2,"quadrate:10, four-side:10, equal-side:10, quad...","plaza:4, town square:2, center:2, market squar...",A massive four - storey square tower is in the...
5234,19562584,square,n,"119,125",1,1,"quadrate:10, four-side:10, equal-side:10, quad...","quadrate:10, four-side:10, equal-side:10, quad...",The most obvious change was the revised front ...
5235,1315823,square,n,"105,111",2,2,"plaza:4, town square:2, center:2, market squar...","plaza:4, town square:2, center:2, market squar...",After the establishment of Serbian rule and th...
5236,1631336,square,n,"25,31",2,2,"plaza:4, town square:2, center:2, market squar...","plaza:4, town square:2, center:2, market squar...","An iron gate circles the square , surrounding ..."
5237,4977925,square,n,"215,222",2,2,"plaza:4, town square:2, center:2, market squar...","plaza:4, town square:2, center:2, market squar...","The rest of the week , not given over to labou..."
5238,7623895,square,n,"71,77",2,2,"plaza:4, town square:2, center:2, market squar...","plaza:4, town square:2, center:2, market squar...",The area in front of the Ajuda Convent was tur...
5239,8084340,square,n,"73,79",2,2,"plaza:4, town square:2, center:2, market squar...","plaza:4, town square:2, center:2, market squar...",Mississauga planned over 60 free events to bri...


In [49]:
word_vectors = 0
context_vectors = 0
clf = 0